In [292]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import r2_score

In [293]:
games = pd.read_csv('../input/nba-games/games.csv')
details = pd.read_csv('../input/nba-games/games_details.csv')
teams = pd.read_csv('../input/nba-games/teams.csv')
players = pd.read_csv('../input/nba-games/players.csv')
ranking = pd.read_csv('../input/nba-games/ranking.csv')

C:\Users\Clay\AppData\Local\Temp\ipykernel_158748\977498853.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  details = pd.read_csv('../input/nba-games/games_details.csv')


In [294]:
def get_labels(ranking):
    temp = ranking.copy(deep=False)
    temp = temp.groupby(['TEAM_ID','SEASON_ID'])['G','W'].max()
    temp = pd.DataFrame(temp)
    temp.reset_index(inplace=True)
    drops = []
    for i in range(len(temp)):
        if temp.iloc[i,1] / 10000 > 2:
            temp.iloc[i,1] = temp.iloc[i,1] % 10000
        else:
            drops.append(i)
            continue;
        if (temp.iloc[i,2] != 82):
            drops.append(i)
    for i in range(len(drops)):
        temp.drop([drops[i]], inplace=True)
    temp.reset_index(inplace=True)
    temp.drop(columns=['index'], inplace=True)
    temp.drop(columns=['G'], inplace=True)
#     temp = pd.merge(temp, ranking, how='left', left_on=['TEAM_ID','STANDINGSDATE'], right_on = ['TEAM_ID','STANDINGSDATE'])
#     temp.drop(columns=['STANDINGSDATE','LEAGUE_ID','SEASON_ID_y','CONFERENCE','TEAM','G','W','L','HOME_RECORD','ROAD_RECORD','RETURNTOPLAY'], inplace=True)
    return temp

In [295]:
labels = get_labels(ranking)
labels

C:\Users\Clay\AppData\Local\Temp\ipykernel_158748\989031293.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = temp.groupby(['TEAM_ID','SEASON_ID'])['G','W'].max()


,TEAM_ID,SEASON_ID,W
0,1610612737,2002,35
1,1610612737,2003,28
2,1610612737,2004,13
3,1610612737,2005,26
4,1610612737,2006,30
...,...,...,...
501,1610612766,2015,48
502,1610612766,2016,36
503,1610612766,2017,36
504,1610612766,2018,39


In [296]:
def get_features(games, details):
    temp = pd.merge(games, details, how='left', left_on=['GAME_ID'], right_on = ['GAME_ID'])
    temp = temp[['TEAM_ID','SEASON','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'
             ,'FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS']]
    temp = temp.groupby(['TEAM_ID','SEASON'])['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'
             ,'FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS'].sum()
    temp = pd.DataFrame(temp)
    next_season = []
    temp.reset_index(inplace=True)
    for i in range(len(temp)):
        next_season.append(temp.iloc[i,1] + 1)
    temp['NEXT_SEASON'] = next_season
    return temp

In [297]:
features = get_features(games, details)
features

C:\Users\Clay\AppData\Local\Temp\ipykernel_158748\2543770804.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = temp.groupby(['TEAM_ID','SEASON'])['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'


,TEAM_ID,SEASON,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,NEXT_SEASON
0,1.610613e+09,2003,2857.0,6609.0,320.764,422.0,1256.0,104.083,1555.0,2004.0,...,2543.0,3548.0,1666.0,633.0,410.0,1292.0,1853.0,7691.0,-1905.0,2004
1,1.610613e+09,2004,2997.0,6821.0,357.705,309.0,997.0,114.160,1456.0,2059.0,...,2387.0,3510.0,1649.0,642.0,350.0,1289.0,2068.0,7759.0,-3983.0,2005
2,1.610613e+09,2005,3196.0,6997.0,358.522,450.0,1205.0,138.741,1804.0,2404.0,...,2427.0,3568.0,1759.0,633.0,417.0,1377.0,2235.0,8646.0,-1950.0,2006
3,1.610613e+09,2006,3105.0,6986.0,369.110,385.0,1161.0,127.166,1865.0,2438.0,...,2528.0,3595.0,1728.0,663.0,480.0,1400.0,2195.0,8460.0,-1950.0,2007
4,1.610613e+09,2007,3457.0,7668.0,382.692,436.0,1250.0,122.879,2129.0,2758.0,...,2885.0,4074.0,2084.0,696.0,536.0,1411.0,2016.0,9479.0,-1165.0,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,1.610613e+09,2018,3495.0,7788.0,413.804,1043.0,2949.0,245.989,1619.0,2046.0,...,2980.0,3824.0,2029.0,625.0,435.0,1038.0,1679.0,9652.0,-330.0,2019
595,1.610613e+09,2019,2615.0,6015.0,297.664,839.0,2397.0,180.742,1135.0,1506.0,...,2249.0,3012.0,1668.0,466.0,297.0,1008.0,1333.0,7204.0,-2260.0,2020
596,1.610613e+09,2020,3210.0,7117.0,352.732,1095.0,2976.0,211.146,1299.0,1729.0,...,2704.0,3559.0,2173.0,640.0,386.0,1160.0,1485.0,8814.0,-899.0,2021
597,1.610613e+09,2021,3685.0,7945.0,398.470,1190.0,3301.0,218.024,1373.0,1859.0,...,2934.0,3877.0,2409.0,747.0,428.0,1129.0,1736.0,9933.0,-380.0,2022


In [298]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [299]:
def get_data(ranking, games, details):
    labels = get_labels(ranking)
    features = get_features(games, details)
    data = pd.merge(labels, features, how='left', left_on=['TEAM_ID','SEASON_ID'], right_on = ['TEAM_ID','NEXT_SEASON'])
    data.drop(columns=['SEASON_ID','SEASON'], inplace=True)
    data.dropna(inplace=True)
    data = swap_columns(data, 'W', 'NEXT_SEASON')
    data = data.astype({'NEXT_SEASON': 'int64'})
    data.rename(columns={'W' : 'NEXT_W'}, inplace=True)
    data.reset_index(inplace=True)
    data.drop(columns=['index'], inplace=True)
    return data

In [300]:
data = get_data(ranking, games, details)
data

C:\Users\Clay\AppData\Local\Temp\ipykernel_158748\989031293.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = temp.groupby(['TEAM_ID','SEASON_ID'])['G','W'].max()
C:\Users\Clay\AppData\Local\Temp\ipykernel_158748\2543770804.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = temp.groupby(['TEAM_ID','SEASON'])['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'


,TEAM_ID,NEXT_SEASON,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,NEXT_W
0,1610612737,2004,2857.0,6609.0,320.764,422.0,1256.0,104.083,1555.0,2004.0,...,2543.0,3548.0,1666.0,633.0,410.0,1292.0,1853.0,7691.0,-1905.0,13
1,1610612737,2005,2997.0,6821.0,357.705,309.0,997.0,114.160,1456.0,2059.0,...,2387.0,3510.0,1649.0,642.0,350.0,1289.0,2068.0,7759.0,-3983.0,26
2,1610612737,2006,3196.0,6997.0,358.522,450.0,1205.0,138.741,1804.0,2404.0,...,2427.0,3568.0,1759.0,633.0,417.0,1377.0,2235.0,8646.0,-1950.0,30
3,1610612737,2007,3105.0,6986.0,369.110,385.0,1161.0,127.166,1865.0,2438.0,...,2528.0,3595.0,1728.0,663.0,480.0,1400.0,2195.0,8460.0,-1950.0,37
4,1610612737,2008,3457.0,7668.0,382.692,436.0,1250.0,122.879,2129.0,2758.0,...,2885.0,4074.0,2084.0,696.0,536.0,1411.0,2016.0,9479.0,-1165.0,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,1610612766,2015,3191.0,7612.0,370.551,551.0,1736.0,154.944,1547.0,2057.0,...,3070.0,3965.0,1833.0,552.0,492.0,1017.0,1691.0,8480.0,-1430.0,48
443,1610612766,2016,3528.0,8122.0,425.323,987.0,2745.0,266.812,1859.0,2352.0,...,3403.0,4246.0,2027.0,691.0,485.0,1140.0,1758.0,9902.0,1195.0,36
444,1610612766,2017,3318.0,7589.0,387.285,866.0,2526.0,188.367,1741.0,2146.0,...,3111.0,3904.0,2024.0,621.0,425.0,1004.0,1486.0,9243.0,-175.0,36
445,1610612766,2018,3386.0,7537.0,390.534,872.0,2379.0,212.096,1738.0,2334.0,...,3078.0,3954.0,1869.0,594.0,393.0,1093.0,1502.0,9382.0,90.0,39


In [303]:
x = data[data.columns.difference(['NEXT_W'])]
y = data[['NEXT_W']]

In [305]:
print(x)

        AST    BLK    DREB    FG3A    FG3M  FG3_PCT     FGA     FGM   FG_PCT  \
0    1666.0  410.0  2543.0  1256.0   422.0  104.083  6609.0  2857.0  320.764   
1    1649.0  350.0  2387.0   997.0   309.0  114.160  6821.0  2997.0  357.705   
2    1759.0  417.0  2427.0  1205.0   450.0  138.741  6997.0  3196.0  358.522   
3    1728.0  480.0  2528.0  1161.0   385.0  127.166  6986.0  3105.0  369.110   
4    2084.0  536.0  2885.0  1250.0   436.0  122.879  7668.0  3457.0  382.692   
..      ...    ...     ...     ...     ...      ...     ...     ...      ...   
442  1833.0  492.0  3070.0  1736.0   551.0  154.944  7612.0  3191.0  370.551   
443  2027.0  485.0  3403.0  2745.0   987.0  266.812  8122.0  3528.0  425.323   
444  2024.0  425.0  3111.0  2526.0   866.0  188.367  7589.0  3318.0  387.285   
445  1869.0  393.0  3078.0  2379.0   872.0  212.096  7537.0  3386.0  390.534   
446  2173.0  386.0  2704.0  2976.0  1095.0  211.146  7117.0  3210.0  352.732   

        FTA  ...   FT_PCT  NEXT_SEASON 

In [306]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()
x = scaler_x.fit_transform(x)
y = scaler_y.fit_transform(y)

In [307]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

In [308]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
seq_len = X_train[0].shape[0]

In [309]:
print(X_train)

tensor([[-0.4207,  0.4727, -0.5396,  ..., -1.1757, -1.3312, -0.0799],
        [ 0.3945,  0.3931, -0.3688,  ...,  0.2184, -1.0995, -1.4637],
        [ 2.5404,  0.0066,  1.5629,  ...,  1.2421, -1.5629,  0.7586],
        ...,
        [ 3.7580,  3.5876,  2.4139,  ...,  1.5540, -0.8679,  1.8766],
        [-1.1231, -0.7437, -0.5200,  ..., -1.1757, -1.3312, -0.3116],
        [ 1.9016,  1.7232,  1.4117,  ...,  2.8116, -0.4045,  1.6721]])


In [310]:
class NBA(Dataset):
    def __init__(self, X_train, y_train):
        super(Dataset, self).__init__()
        self.X_train = X_train
        self.y_train = y_train

    def __len__(self):
        return len(self.X_train)
    
    def __getitem__(self, idx):
        x = self.X_train[idx]
        y = self.y_train[idx]
        return x, y

In [311]:
hidden_size = 512
num_layers = 2
learning_rate = 0.001
batch_size = 64
epoch_size = 10

In [312]:
train_dataset = NBA(X_train, y_train)
test_dataset = NBA(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [313]:
class RNN(nn.Module):
    def __init__(self, input_feature_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.lstm = nn.LSTM(input_feature_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [316]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
rnn = RNN(21, hidden_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.0001)

In [317]:
rnn.train()
for epoch in range(epoch_size):
    loss = 0.0

    for batch_idx, data in enumerate(train_loader):
        inputs, targets = data
        inputs.to(device)
        targets.to(device)

        optimizer.zero_grad()

        outputs = rnn(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        loss += loss.item()
        if batch_idx % 100 == 99:
            print(f'[{epoch + 1}, {batch_idx + 1:5d}] loss: {loss / 100:.3f}')
            loss = 0.0

In [318]:
prediction = []
ground_truth = []

rnn.eval()
with torch.no_grad():
    for data in test_loader:
        inputs, targets = data
        inputs = inputs.to(device)

        ground_truth += targets.flatten().tolist()
        out = rnn(inputs).detach().cpu().flatten().tolist()
        prediction += out

In [319]:
print(type(prediction))
print(prediction)

<class 'list'>
[-0.2017410844564438, -0.09019644558429718, -0.19960236549377441, -0.35338011384010315, -0.35115259885787964, -0.29297181963920593, 0.006201453506946564, -0.19345712661743164, -0.19520153105258942, -0.04578936845064163, 0.27717193961143494, 0.13694722950458527, 0.30836722254753113, -0.09135600924491882, -0.2146719992160797, 0.3409160375595093, 0.1587938368320465, -0.2703396677970886, 0.2040511667728424, -0.22338661551475525, -0.2641177475452423, -0.29654523730278015, 0.10499289631843567, -0.1170753613114357, -0.129383385181427, -0.0434286892414093, -0.21575307846069336, -0.17054498195648193, -0.20506595075130463, 0.2409622073173523, -0.049079060554504395, -0.10542075335979462, 0.3748258054256439, -0.33350035548210144, -0.3628946840763092, -0.2841795086860657, -0.2555449306964874, -0.3283269703388214, -0.31811100244522095, -0.2972012460231781, -0.2824123799800873, -0.04288310930132866, -0.24759721755981445, -0.272788941860199, -0.2355758249759674, -0.05491926521062851, -0

In [320]:
# prediction = scaler_y.inverse_transform(prediction)
# ground_truth = scaler_y.inverse_transform(ground_truth)

In [321]:
r2score = r2_score(prediction, ground_truth)
print(r2score)

-14.676363440939435


In [322]:
for i in range(len(prediction)):
    print("Prediction: ", prediction[i], ", ground truth: ", ground_truth[i])
    print(prediction[i] - ground_truth[i])

Prediction:  -0.2017410844564438 , ground truth:  -1.3704775584469473
1.1687364739905035
Prediction:  -0.09019644558429718 , ground truth:  -0.7267329441022363
0.6365364985179391
Prediction:  -0.19960236549377441 , ground truth:  1.0435647453457184
-1.2431671108394928
Prediction:  -0.35338011384010315 , ground truth:  -0.9681371744815029
0.6147570606413998
Prediction:  -0.35115259885787964 , ground truth:  1.2045008989318962
-1.5556534977897758
Prediction:  -0.29297181963920593 , ground truth:  -0.08298832975752554
-0.2099834898816804
Prediction:  0.006201453506946564 , ground truth:  -1.3704775584469473
1.3766790119538939
Prediction:  -0.19345712661743164 , ground truth:  0.8826285917595407
-1.0760857183769723
Prediction:  -0.19520153105258942 , ground truth:  -0.6462648673091476
0.45106333625655815
Prediction:  -0.04578936845064163 , ground truth:  0.2388839774148299
-0.28467334586547155
Prediction:  0.27717193961143494 , ground truth:  0.5607562845871853
-0.28358434497575036
Predict